In [1]:
import os

import catchment_delineation as cd
import numpy as np
from pysheds.grid import Grid
from WBT.whitebox_tools import WhiteboxTools

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


# Terrian conditioning for Glomma at 10 m resolution

The `fill_depressions` function in PySheds calls `skimage.morphology.reconstruction`. This seems to fail for large arrays - see the issue [here](https://github.com/scikit-image/scikit-image/issues/6277). The only dataset large enough to cause problems in the catchment delineation workflow is Glomma at 10 m resolution. Until the issue with Sci-Kit Image is resolved, this notebook used Whitebox Tools instead of PySheds to fill depressions. This takes longer, and it's not completely consistent with the workflow for other catchments, but it seems to produce comparable results.

**Note:** This notebook must be run on a machine with a lot of memory (e.g. 240 GB). Expect the processing to take about 4 to 5 hours.

In [2]:
wbt = WhiteboxTools()
wbt.set_verbose_mode(True)
wbt.set_compress_rasters(True)

work_dir = r"/home/jovyan/shared/01_datasets/spatial"
wbt.set_working_dir(work_dir)

['./whitebox_tools', '-v']


In [3]:
def condition_dem(
    raw_dem_path,
    intermed_dem_path,
    fill_dem_path,
    fdir_path,
    facc_path,
    dem_dtype=np.int16,
    dem_ndv=-32767,
    burn_streams=False,
    shapes=None,
    sigma=None,
    dz=None,
    max_iter=1e9,
    eps=1e-12,
):
    """ALTERNATIVE TO THE FUNCTION IN NIVAPY. USES WHITEBOX TOOLS TO FILL
    DEPRESSIONS.
    
    Burns streams into DEM, fills pits and depressions, and calculates flow
    direction and accumulation. The filled DEM is converted to the specificed
    dtype and saved. Flow direction and accumulation rasters are also saved.

    Args
        raw_dem_path:  Str. Path to raw DEM to be processed
        fill_dem_path: Str. Output path for filled (and optionally burned) DEM
        fdir_path:     Str. Output path for flow direction
        facc_path:     Str. Output path for flow accumulation
        dem_dtype:     Obj. Numpy data type for output DEM
        dem_ndv:       Float or int. NoData value for output DEM
        burn_streams:  Bool. Whether to burn streams
        shapes:        Tuple of tuples. Stream shapes to burn. Only valid if
                       'burn_streams' is True
        sigma:         Float. Std. dev. for Gaussian blur applied to streams.
                       Only valid if 'burn_streams' is True
        dz:            Float or int. Stream burn depth. Only valid if
                       'burn_streams' is True
        max_iter:      Int. Default 1e9. Maximum iterations for filling flats
        eps:           Float. Default 1e-12. Parameter for flat-filling
                       algorithm. See example notebook

    Returns
        Tuple of PySheds 'Raster' arrays (dem, fdir, facc).
    """
    # Check user input
    if burn_streams:
        assert None not in (
            shapes,
            sigma,
            dz,
        ), "'shapes', 'sigma' and 'dz' are all required when 'burn_streams' is True."
    else:
        assert all(
            v is None for v in (shapes, sigma, dz)
        ), "'shapes', 'sigma' and 'dz' are not required when 'burn_streams' is False."

    dirmap = (1, 2, 3, 4, 5, 6, 7, 8)
    grid = Grid.from_raster(raw_dem_path)
    dem = grid.read_raster(raw_dem_path).astype(np.float32)
    ndv = dem.nodata.copy()

    # 'fill_depressions' isn't designed for NoData (see
    #     https://github.com/scikit-image/scikit-image/issues/4078)
    # Either set NoData and values < 0 to zero or -dz, depending on
    # whether streams are being burned. This forces all cells to drain
    # to the edge of the grid
    mask = dem == ndv
    if burn_streams:
        dem[mask] = -dz
        dem[dem < 0] = -dz
    else:
        dem[mask] = 0
        dem[dem < 0] = 0

    if burn_streams:
        dem = cd.burn_stream_shapes(grid, dem, shapes, sigma, dz)
    dem = grid.fill_pits(dem, nodata_in=np.nan, nodata_out=np.nan)
    #    dem = grid.fill_depressions(dem, nodata_in=np.nan, nodata_out=np.nan)

    dem = dem.astype(np.float64)
    cd.pysheds_array_to_raster(
        dem, raw_dem_path, intermed_dem_path, np.float64, dem_ndv
    )

    wbt.fill_depressions(
        intermed_dem_path,
        fill_dem_path,
        fix_flats=False,
        flat_increment=None,
        max_depth=None,
    )

    grid = Grid.from_raster(fill_dem_path)
    dem = grid.read_raster(fill_dem_path).astype(np.float64)

    dem = grid.resolve_flats(
        dem, max_iter=max_iter, eps=eps, nodata_in=np.nan, nodata_out=np.nan
    )

    npits = grid.detect_pits(dem).sum()
    nflats = grid.detect_flats(dem).sum()
    if (npits > 0) or (nflats > 0):
        fill_fname = os.path.split(fill_dem_path)[1]
        msg = f"        {fill_fname} has {npits} pits and {nflats} flats."
        print(msg)

    # Flow dir and accum
    fdir = grid.flowdir(
        dem, routing="d8", dirmap=dirmap, nodata_in=np.nan, nodata_out=0
    )
    facc = grid.accumulation(
        fdir, routing="d8", dirmap=dirmap, nodata_in=0, nodata_out=0
    )

    # Save results to disk
    if np.issubdtype(dem_dtype, np.integer):
        dem = np.rint(dem)
    dem[np.isnan(dem)] = dem_ndv
    dem = dem.astype(dem_dtype)
    cd.pysheds_array_to_raster(dem, raw_dem_path, fill_dem_path, dem_dtype, dem_ndv)

    fdir = fdir.astype(np.int16)
    cd.pysheds_array_to_raster(fdir, raw_dem_path, fdir_path, np.int16, 0)

    facc = facc.astype(np.uint32)
    cd.pysheds_array_to_raster(facc, raw_dem_path, facc_path, np.uint32, 0)

    return (dem, fdir, facc)

In [4]:
%%time

import pickle

with open(r"/home/jovyan/shared/01_datasets/spatial/streams.pkl", "rb") as f:
    shapes = pickle.load(f)

CPU times: user 17.9 s, sys: 1.32 s, total: 19.2 s
Wall time: 19.2 s


In [5]:
no_data_val = -32767
dem_dtype = np.float32
sigma = 3
dz = 20
max_iter = 1e9
eps = 1e-12

raw_dem_path = r"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_10m/by_vassom/dtm/vassom_002_10m_dtm.tif"
intermed_dem_path = r"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_10m/by_vassom/glomma/intermed.tif"
fill_dem_path = r"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_10m/by_vassom/dtm_fill_burn/vassom_002_10m_burn_fill.tif"
fdir_path = r"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_10m/by_vassom/flow_direction/vassom_002_10m_fdir.tif"
facc_path = r"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_10m/by_vassom/flow_accumulation/vassom_002_10m_facc.tif"

condition_dem(
    raw_dem_path,
    intermed_dem_path,
    fill_dem_path,
    fdir_path,
    facc_path,
    dem_dtype=dem_dtype,
    dem_ndv=no_data_val,
    burn_streams=True,
    shapes=shapes,
    sigma=sigma,
    dz=dz,
    max_iter=max_iter,
    eps=eps,
)

./whitebox_tools --run="FillDepressions" --wd="/home/jovyan/shared/01_datasets/spatial" --dem='/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_10m/by_vassom/glomma/intermed.tif' --output='/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_10m/by_vassom/glomma/fill.tif' --compress_rasters

******************************
* Welcome to FillDepressions *
* Powered by WhiteboxTools   *
* www.whiteboxgeo.com        *
******************************
Reading data...
Finding pit cells: 2%
Finding pit cells: 5%
Finding pit cells: 7%
Finding pit cells: 10%
Finding pit cells: 12%
Finding pit cells: 15%
Finding pit cells: 17%
Finding pit cells: 20%
Finding pit cells: 22%
Finding pit cells: 25%
Finding pit cells: 27%
Finding pit cells: 30%
Finding pit cells: 32%
Finding pit cells: 35%
Finding pit cells: 37%
Finding pit cells: 40%
Finding pit cells: 42%
Finding pit cells: 45%
Finding pit cells: 47%
Finding pit cells: 50%
Finding pit cells: 52%
Finding pit cells: 55%
Finding pit c

(Raster([[-2.e+01, -2.e+01, -2.e+01, ..., -2.e+01, -2.e+01, -2.e+01],
         [-2.e+01, -2.e+01, -2.e+01, ..., -2.e+01, -2.e+01, -2.e+01],
         [-2.e+01, -2.e+01, -2.e+01, ..., -2.e+01, -2.e+01, -2.e+01],
         ...,
         [-2.e+01, -2.e+01, -2.e+01, ...,  7.e-12,  4.e-12,  2.e-12],
         [-2.e+01, -2.e+01, -2.e+01, ...,  5.e-12,  4.e-12,  2.e-12],
         [-2.e+01, -2.e+01, -2.e+01, ...,  2.e-12,  2.e-12,  2.e-12]],
        dtype=float32),
 Raster([[0, 0, 0, ..., 0, 0, 0],
         [0, 1, 1, ..., 1, 1, 0],
         [0, 7, 1, ..., 3, 3, 0],
         ...,
         [0, 7, 5, ..., 3, 3, 0],
         [0, 5, 5, ..., 5, 3, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=int16),
 Raster([[   0,    1,    2, ...,    1,    1,    0],
         [   0,    1,    2, ...,    1,    1,    0],
         [ 190,  190,    1, ...,    1, 2329, 2329],
         ...,
         [   1,    1,    1, ...,    1,    2,    2],
         [   0,    1,    2, ..., 6362,    1,    1],
         [   0,    1,    2, ..., 6